# My ETF Portfolio Holdings
> A visual observation into my ETF portfolio

- toc: false
- badges: false
- comments: true
- categories: [Finance]
- image: images/chart-preview.png


In this notebook I wanted to visualy display 3 ETFs I own and thier underlying holdings. The 3 ETFS include the NASDAQ 100 (ASX:NDQ), Asian technology tigers (ASX:ASIA) and a clean energy index (ASX:CLNE). The colour shows the percentage change of my portfolio. 

In [1]:
#hide
#@title IMPORTS
import pandas as pd
import numpy as np
!pip install yfinance
import yfinance as yf
import seaborn as sns
import requests
from bs4 import BeautifulSoup
!pip install plotly==4.5.2
import plotly
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
 
import nltk
from nltk.sentiment.vader import SentimentIntensityAnalyzer as SID
nltk.download('vader_lexicon')
from datetime import timedelta, date

     |████████████████████████████████| 6.3MB 9.0MB/s 
  Created wheel for yfinance: filename=yfinance-0.1.59-py2.py3-none-any.whl size=23455 sha256=67c36f63876551a4c8722af23e36a13b5f9fcfe1f1eef9bf1b36384014ab4a11
  Stored in directory: /root/.cache/pip/wheels/f8/2a/0f/4b5a86e1d52e451757eb6bc17fd899629f0925c777741b6d04
Successfully built yfinance
  Found existing installation: lxml 4.2.6
    Uninstalling lxml-4.2.6:
      Successfully uninstalled lxml-4.2.6
     |████████████████████████████████| 7.1MB 6.0MB/s 
  Found existing installation: plotly 4.4.1
    Uninstalling plotly-4.4.1:
      Successfully uninstalled plotly-4.4.1
[nltk_data] Downloading package vader_lexicon to /root/nltk_data...


/usr/local/lib/python3.7/dist-packages/nltk/twitter/__init__.py:20: UserWarning:

The twython library has not been installed. Some functionality from the twitter package will not be available.



In [2]:
#hide
#@title HELPER FUNCTIONS
def get_pc(symbol):
    try:
      ticker = yf.Ticker(symbol)
      todays_data = ticker.history(period='2d')
      # print(todays_data['Close'])
      pc = 100*(todays_data['Close'][1]-todays_data['Close'][0])/todays_data['Close'][0]
      # print(symbol,pc)
    except: 
      pc = 0
  
    return pc

def get_current_price(symbol):
    try:
      ticker = yf.Ticker(symbol)
      todays_data = ticker.history(period='1d')
      c = todays_data['Close'][0]
    except: 
      c = 0
  
    return c

In [3]:
#hide
#@title MAYBE FUNCTIONS

def get_inverse_exchange(symbol='AUDUSD=X'):
    try:
      ticker = yf.Ticker(symbol)
      todays_data = ticker.history(period='1d')
      close = 1/todays_data['Close'][0]
    except:
      close = 0

    return close

In [4]:
#hide
#@title NDQ DATA
NDQ = 'https://www.betashares.com.au/files/csv/NDQ_Portfolio_Holdings.csv'
 
df = pd.read_csv(NDQ, header = 6)
df = df[['Ticker', 'Name', 'Sector', 'Currency', 'Weight (%)']].iloc[:-4,:]
df['% Change'] = [get_pc(x[:-3]) for x in df['Ticker']]
df['Ticker'] = [x[:-3] for x in df['Ticker']]
NDQ = df.copy()

- NQU1 IN: No data found, symbol may be delisted
- : No data found for this date range, symbol may be delisted
- : No data found for this date range, symbol may be delisted


In [5]:
#hide
#@title ASIA DATA
ASIA = 'https://www.betashares.com.au/files/csv/ASIA_Portfolio_Holdings.csv'
 
df = pd.read_csv(ASIA, header = 6)
df = df[['Ticker', 'Name', 'Sector','Currency', 'Weight (%)']].iloc[:-4,:]
exchanges = pd.DataFrame([x[-2:] for x in df['Ticker']],columns=['x'])['x'].unique()
ex = [' KP', ' UN', ' HK', ' UW', ' TT', ' UQ', ' RW', ' SD', ' WD', ' KD', ' UD']
rep = ['.KS', '', '.HK', '', '.TW', 'N.MX', '','','', '','','']
# if ends in .HK add 0 the front
 
for x,y in zip(ex,rep):
  df['Ticker'] = df['Ticker'].str.replace(x,y, regex=False)
 
ASIA = df.copy()
 
df[df['Ticker'].str.contains('.HK')] = ('0' + df[df['Ticker'].str.contains('.HK')].astype(str))
 
ASIA['Ticker'] = df['Ticker']
 
r1 = ['03690.HK', 'GDSN.MX', '03888.HK', '6488.TW', '3105.TW', '5347.TW']
r2 = ['3690.HK', 'GDS', '3888.HK', '6488.TWO', '3105.TWO', '5347.TWO']
 
for x,y in zip(r1,r2):
  ASIA['Ticker'] = ASIA['Ticker'].str.replace(x,y, regex=False)
 
ASIA['% Change'] = [get_pc(x) for x in ASIA['Ticker']]

- 01024.HK: No data found, symbol may be delisted
- KRW: No data found for this date range, symbol may be delisted
- HKD: No data found for this date range, symbol may be delisted
- TWD: No data found for this date range, symbol may be delisted


In [6]:
#hide
#@title CLNE DATA
ICLN = 'https://www.ishares.com/us/products/239738/ishares-global-clean-energy-etf/1467271812596.ajax?fileType=csv&fileName=ICLN_holdings&dataType=fund'

ICLN  = pd.read_csv(ICLN, header = 9, error_bad_lines=False)
ICLN['Ticker_'] = ICLN['Ticker']
CLNE = "https://www.vaneck.com.au/etf/equity/CLNE/holdings/"
df = pd.read_html(CLNE)
df = df[1].iloc[:-2,:]
ex = [' US', ' AV', ' SM', ' NZ', ' HK', ' DC', ' PL', ' NO', ' CN', ' GR', ' KS',
       ' FP', ' IT', ' SS', ]
df['Ticker_'] = df['Ticker']

for x,y in zip(ex,rep):
  df['Ticker_'] = df['Ticker_'].str.replace(x,'', regex=False)


df = pd.merge(df,ICLN[['Ticker_','Sector']], on = ['Ticker_'])
df = df[['Ticker', 'Holding name', 'Sector', '% of Fund net assets']]
df.columns = ['Ticker', 'Name','Sector', 'Weight (%)']

# exchanges = pd.DataFrame([x[-2:] for x in df['Ticker']],columns=['x'])['x'].unique()
ex = [' US', ' AV', ' SM', ' NZ', ' HK', ' DC', ' PL', ' NO', ' CN', ' GR', ' KS',
       ' FP', ' IT', ' SS', ]
rep = ['', '.VI', '.MC', '.NZ', '.HK', '.CO', '.LS','.OL','.TO', '.DE','.KS',
       '.NX','.TA','.ST']

for x,y in zip(ex,rep):
  df['Ticker'] = df['Ticker'].str.replace(x,y, regex=False)

r1 = ['NEOEN.NX']
r2 = ['NEOEN.PA']

for x,y in zip(r1,r2):
  df['Ticker'] = df['Ticker'].str.replace(x,y, regex=False)

CLNE = df.copy()

df[df['Ticker'].str.contains('.HK')] = ('0' + df[df['Ticker'].str.contains('.HK')].astype(str))

CLNE['Ticker'] = df['Ticker']

CLNE['% Change'] = [get_pc(x) for x in CLNE['Ticker']]
CLNE['Weight (%)'] = CLNE['Weight (%)']/100

In [7]:
#hide
#@title PORTFOLIO
ndq_v = 190*get_current_price('NDQ.AX').astype(int)
asia_v = 493*get_current_price('ASIA.AX').astype(int)
clne_v = 269*get_current_price('CLNE.AX').astype(int)
NDQ['ETF'] = 'NDQ'
ASIA['ETF'] = 'ASIA'
CLNE['ETF'] = 'CLNE'
 
NDQ['AUD holding'] = NDQ['Weight (%)']*ndq_v
ASIA['AUD holding'] = ASIA['Weight (%)']*asia_v
CLNE['AUD holding'] = CLNE['Weight (%)']*clne_v
NDQ['Total Weight (%)'] = round((NDQ['AUD holding'] / (ndq_v + asia_v + clne_v))*100,2)
ASIA['Total Weight (%)'] = round((ASIA['AUD holding'] / (ndq_v + asia_v + clne_v))*100,2)
CLNE['Total Weight (%)'] = round((CLNE['AUD holding'] / (ndq_v + asia_v + clne_v))*100,2)
 
PORT = pd.concat([ ASIA,NDQ, CLNE])
PORT['AUD holding'] = PORT['AUD holding'].fillna(0)
PORT['AUD holding'] = pd.to_numeric(PORT['AUD holding'])
 
# add and duplicates
x = PORT.groupby('Ticker').sum().reset_index()
PORT = pd.merge(PORT[['Ticker', 'Name', 'ETF', 'Sector','Currency', 'Weight (%)', '% Change']].drop_duplicates(subset='Ticker', keep="first"),
                x[['Ticker','AUD holding','Total Weight (%)']], on='Ticker', how='outer')
PORT = PORT.rename(columns={"Weight (%)": "ETF Weight (%)"})
PORT['Sector'] = PORT['Sector'].replace({np.nan: 'Other'})
 
PORT = PORT.sort_values("AUD holding", axis = 0, ascending = False) 
 
PORT.reset_index(drop=True).style.format({'AUD holding': '{:.2f}',
                                          '% Change': '{:.2f}',
                                          'Total Weight (%)': '{:.2f}',
                                          'ETF Weight (%)': '{:.2f}'}).background_gradient(subset=['Total Weight (%)','% Change'], 
                                                                                           cmap='RdYlGn')

,Ticker,Name,ETF,Sector,Currency,ETF Weight (%),% Change,AUD holding,Total Weight (%)
0,AAPL,APPLE INC,NDQ,Information Technology,USD,0.11,1.96,651.33,4.65
1,MSFT,MICROSOFT CORP,NDQ,Information Technology,USD,0.10,2.23,583.16,4.16
2,0700.HK,TENCENT HOLDINGS LTD,ASIA,Communication Services,HKD,0.10,-1.63,548.32,3.92
3,TSM,TAIWAN SEMICONDUCTOR MANUFACTU,ASIA,Information Technology,USD,0.10,0.41,547.39,3.91
4,BABA,ALIBABA GROUP HOLDING LTD,ASIA,Consumer Discretionary,USD,0.10,-1.86,542.08,3.87
5,005930.KS,SAMSUNG ELECTRONICS CO LTD,ASIA,Information Technology,KRW,0.10,-0.12,528.33,3.77
6,AMZN,AMAZON.COM INC,NDQ,Consumer Discretionary,USD,0.08,2.27,493.79,3.53
7,3690.HK,MEITUAN,ASIA,Consumer Discretionary,HKD,0.07,-5.12,391.67,2.80
8,PDD,PINDUODUO INC,ASIA,Consumer Discretionary,USD,0.05,-2.17,263.84,1.89
9,JD,JD.COM INC,ASIA,Consumer Discretionary,USD,0.04,-1.56,249.67,1.79


In [24]:

#hide_input
#@title PORT TREEMAP
from datetime import datetime
%matplotlib inline
 
PORT = PORT.iloc[:-1,:] 
PORT = PORT.replace({0: 0.0001})
# df = px.data.tips()
PORT["PORT"] = "Portfolio"
fig = px.treemap(PORT, path=['PORT','Sector','ETF','Name'],
                 values='Total Weight (%)',
                 color='% Change',
                 color_continuous_scale='RdYlGn',
                 color_continuous_midpoint=0,
                 hover_data=['AUD holding'],
                 range_color = [-3,3]
                 )
 
fig.update_layout({
'plot_bgcolor': 'rgba(0, 0, 0, 0)',
'paper_bgcolor': 'rgba(0, 0, 0, 0)',
})
 
 
fig.update_layout(#yaxis=dict(scaleanchor='y', constrain='domain'),
                  width=747, 
                  height=750 )
 
fig.update_layout(legend_title_text='% Change')
fig.update_layout(title=f"Portfolio on {datetime.today().strftime('%Y-%m-%d')}")
 
 
fig.show()

In [9]:
#hide
#@title PORT PRICE DIFF
print('The portfolio price difference should be: $',
      round(((PORT['% Change']/100)*PORT['AUD holding']).sum(),2))

The portfolio price difference should be: $ -7.28


In [10]:

#hide_input
#@title SEE: X ETF
# PORT.iloc[:,5].sum() 
# print(ndq_v , asia_v , clne_v)
# CLNE['AUD holding'] = CLNE['Weight (%)']*clne_v
# df= CLNE.copy()
# df = df.rename(columns={"Weight (%)": "ETF Weight (%)"})
# df.reset_index(drop=True).style.format({'AUD holding': '{:.2f}',
#                                           'Price Change': '{:.2f}',
#                                           'ETF Weight (%)': '{:.3f}'}).background_gradient(subset=['ETF Weight (%)','Price Change'], 
#                                                                                            cmap='Reds')

In [11]:

#hide
#@title NDQ SENTIMENT 
ndq = NDQ.iloc[:-3,:].copy()
ndq = ndq.set_index('Ticker')
 
header = {
  "User-Agent": "Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/50.0.2661.75 Safari/537.36",
  "X-Requested-With": "XMLHttpRequest"
}
 
for tick in ndq.index.tolist(): 
  link = f'https://finviz.com/quote.ashx?t={tick}'
 
  r = requests.get(link, headers=header)
  soup = BeautifulSoup(r.text, "html5lib")
 
  table = soup.find(id="news-table")
  df = pd.read_html(str(table))[0]
  df.columns = ['Date', 'Headline']
  df['Time'] = [x[-7:] for x in df['Date']]
  df['Date'] = [x[:-7] for x in df['Date']]
 
  # Replace space with nan and use ffill()
  df['Date'] = df['Date'].replace(r'^\s*$', np.nan, regex=True).ffill()
 
  # # Put the columns back and convert to datetime
  df['Date'] =  pd.to_datetime(df['Date'])
  # # Drop the time column
  # del(df['Time'])
  sid = SID()
  df['Sentiment Score'] = [sid.polarity_scores(str(text))['compound'] for text in df['Headline']]
  today = date.today()
  today = today.strftime("%Y-%m-%d")
  end = date.today() - timedelta(days=100)
  end = end.strftime("%Y-%m-%d")
 
  df = df[(df['Date'] > end) & (df['Date'] < today)]
  ndq.loc[tick,'Sentiment Score'] = df['Sentiment Score'].mean()

In [12]:
#hide
#@title  
ndq = ndq.rename(columns={"Weight (%)": "ETF Weight (%)"})
print('Total weighted sentiment:',round((ndq['ETF Weight (%)']*ndq['Sentiment Score']).sum(),2))
ndq.reset_index().style.format({'Sentiment Score': '{:.2f}',
                                'ETF Weight (%)': '{:.3f}',
                                          '% Change': '{:.2f}'}).background_gradient(subset=['Sentiment Score','% Change'], cmap='Reds')

Total weighted sentiment: 0.08


,Ticker,Name,Sector,Currency,ETF Weight (%),% Change,ETF,AUD holding,Total Weight (%),Sentiment Score
0,AAPL,APPLE INC,Information Technology,USD,0.111,1.96,NDQ,651.333870,4.650000,0.10
1,MSFT,MICROSOFT CORP,Information Technology,USD,0.099,2.23,NDQ,583.163010,4.160000,0.11
2,AMZN,AMAZON.COM INC,Consumer Discretionary,USD,0.084,2.27,NDQ,493.788150,3.530000,0.09
3,FB,FACEBOOK INC,Communication Services,USD,0.040,0.09,NDQ,237.013600,1.690000,0.07
4,GOOG,ALPHABET INC,Communication Services,USD,0.039,1.86,NDQ,232.307490,1.660000,0.06
5,TSLA,TESLA INC,Consumer Discretionary,USD,0.038,0.14,NDQ,225.710690,1.610000,0.10
6,NVDA,NVIDIA CORP,Information Technology,USD,0.037,1.36,NDQ,216.657760,1.550000,0.16
7,GOOGL,ALPHABET INC,Communication Services,USD,0.036,2.30,NDQ,210.096300,1.500000,0.06
8,PYPL,PAYPAL HOLDINGS INC,Information Technology,USD,0.025,0.43,NDQ,144.687850,1.030000,0.15
9,ADBE,ADOBE INC,Information Technology,USD,0.020,1.43,NDQ,120.303250,0.860000,0.07


In [13]:
#hide
#@title Median Sentiment in News and Blogs

header = {
  "User-Agent": "Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/50.0.2661.75 Safari/537.36",
  "X-Requested-With": "XMLHttpRequest"
}


link = 'https://finviz.com/news.ashx'

r = requests.get(link, headers=header)
soup = BeautifulSoup(r.text, "html5lib")

table = soup.find_all('tr')
df1 = pd.read_html(str(table))[2]
df1 = df1.iloc[:,1:3]
df1.columns = ['Time', 'Headline']
df1['Sentiment Score'] = [sid.polarity_scores(str(text))['compound'] for text in df1['Headline']]
print('The median sentiment in the past 48 hours via news is:', round(df1[df1['Sentiment Score']!=0]['Sentiment Score'].median(),3))
df2 = pd.read_html(str(table))[3]
df2 = df2.iloc[:,1:3]
df2.columns = ['Time', 'Headline']
df2['Sentiment Score'] = [sid.polarity_scores(str(text))['compound'] for text in df2['Headline']]
print('The median sentiment in the past 48 hours via blogs is:', round(df2[df2['Sentiment Score']!=0]['Sentiment Score'].median(),3))

# soup.find_all('tr')

The median sentiment in the past 48 hours via news is: 0.202
The median sentiment in the past 48 hours via blogs is: -0.077


## Observing the seasonality of the NASDAQ

In [14]:
#hide_input
MS = pd.DataFrame()
tick = '^IXIC'
ticker = yf.Ticker(tick)
df = ticker.history(period='max',interval='1mo')
df = df.dropna()
df['PC'] = ((df['Close'] - df['Open']) / df['Open'])*100
df['Month'] = df.index.month_name()

MS['Average Return (%)'] = df.groupby(['Month'])['PC'].mean().round(2)
MS['Median Return (%)'] = df.groupby(['Month'])['PC'].median().round(2)
MS['Standard Deviation (%)'] = df.groupby(['Month'])['PC'].std().round(2)
MS['Minimum Return (%)'] = df.groupby(['Month'])['PC'].min().round(2)
MS['Maximum Return (%)'] = df.groupby(['Month'])['PC'].max().round(2)
MS['Number of Observations'] = df.groupby(['Month'])['PC'].count()
MS = MS.sort_values(by='Month')

new_order = ['January', 'February', 'March', 'April', 'May', 'June', 'July', 'August', 'September', 'October', 'November', 'December']

MS = MS.reindex(new_order, axis=0)
MS.index.name = None
# print(' '*50, tick, 'Seasonality')
MS.style.format({'Average Return (%)': '{:.2f}%',
                 'Median Return (%)': '{:.2f}%', 
                 'Standard Deviation (%)': '{:.2f}%',
                  'Minimum Return (%)': '{:.2f}%', 
                  'Maximum Return (%)': '{:.2f}%'
                 }).background_gradient(subset=[ 'Median Return (%)'], 
                                                                                           cmap='RdYlGn')

,Average Return (%),Median Return (%),Standard Deviation (%),Minimum Return (%),Maximum Return (%),Number of Observations
January,2.23%,2.74%,6.18%,-9.95%,13.52%,37
February,0.66%,0.96%,6.81%,-22.36%,18.57%,37
March,0.56%,1.20%,5.06%,-13.45%,12.72%,37
April,1.77%,1.65%,6.07%,-14.11%,19.17%,37
May,1.29%,2.94%,5.53%,-13.47%,10.79%,37
June,0.89%,0.61%,4.98%,-9.31%,14.23%,37
July,0.81%,1.11%,5.08%,-8.86%,10.48%,38
August,0.13%,1.32%,6.35%,-19.81%,11.84%,36
September,-0.58%,0.40%,6.55%,-16.84%,12.25%,36
October,0.71%,2.68%,8.07%,-27.54%,13.33%,36


In [15]:

#hide_input
# MS = pd.DataFrame()
# tick = 'ASIA.AX'
# ticker = yf.Ticker(tick)
# df = ticker.history(period='30y',interval='1d')
# df = df.dropna()
# df['PC'] = ((df['Close'] - df['Open']) / df['Open'])*100
# df['Day'] = df.index.day_name()

# MS['Average Return (%)'] = df.groupby(['Day'])['PC'].mean().round(2)
# MS['Median Return (%)'] = df.groupby(['Day'])['PC'].median().round(2)
# MS['Standard Deviation (%)'] = df.groupby(['Day'])['PC'].std().round(2)
# MS['Minimum Return (%)'] = df.groupby(['Day'])['PC'].min().round(2)
# MS['Maximum Return (%)'] = df.groupby(['Day'])['PC'].max().round(2)
# MS['Number of Observations'] = df.groupby(['Day'])['PC'].count()
# MS = MS.sort_values(by='Day')

# new_order = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']

# MS = MS.reindex(new_order, axis=0)

# print(' '*50, tick, 'Seasonality')
# MS.style.format({'Average Return (%)': '{:.2f}%',
#                  'Median Return (%)': '{:.2f}%', 
#                  'Standard Deviation (%)': '{:.2f}%',
#                   'Minimum Return (%)': '{:.2f}%', 
#                   'Maximum Return (%)': '{:.2f}%'
#                  }).background_gradient(subset=['Average Return (%)', 'Median Return (%)', 'Standard Deviation (%)',
#        'Minimum Return (%)', 'Maximum Return (%)'], 
#                                                                                            cmap='RdYlGn')

In [16]:

#hide_input
#@title A200 DATA
# A200 = 'https://www.betashares.com.au/files/csv/A200_Portfolio_Holdings.csv'
 
# df = pd.read_csv(A200, header = 6)
# df = df[['Ticker', 'Name', 'Currency', 'Weight (%)']].iloc[:-4,:]
# ex = [' AT']
# rep = ['.AX']
 
# for x,y in zip(ex,rep):
#   df['Ticker'] = df['Ticker'].str.replace(x,y, regex=False)
 
# df = df.iloc[:-2,:]
# df['Price Change'] = [get_pc(x) for x in df['Ticker']]
# df['Ticker'] = [x[:-3] for x in df['Ticker']]
# # A200 = df.copy()
 
# df = df.rename(columns={"Weight (%)": "ETF Weight (%)"})
# df.reset_index(drop=True).style.format({'AUD holding': '{:.2f}',
#                                           'Price Change': '{:.2f}',
#                                           'ETF Weight (%)': '{:.3f}'}).background_gradient(subset=['ETF Weight (%)','Price Change'], cmap='RdYlGn')

In [17]:

#hide_input
# !pip install twitterscraper
# from twitterscraper import query_tweets
# import datetime as dt


In [18]:

#hide_input
# # query_tweets('@TradeWithAlerts', limit=1000)

# begin_date=dt.date(2021,3,1)
# end_date=dt.date(2021,3,21)

# limit=500
# lang='english'

# tweets=query_tweets('@TradeWithAlerts',begindate=begin_date,enddate=end_date,limit=limit,lang=lang)
# df=pd.DataFrame(t.__dict__ for t in tweets)
# # df1 = df[['text']]
# # export_csv = df1.to_csv (r'C:/Users/usama/Desktop/123.csv', index = None, header=True)
# # df1

In [19]:

#hide_input
# !pip install twitter-scraper
# from twitter_scraper import get_tweets

In [20]:

#hide_input
# for tweet in get_tweets('jack', pages=2):
#   print(tweet['text'])

In [21]:

#hide_input
# tickers = NDQ['Ticker'].tolist()
# weight = NDQ['Weight (%)'].tolist()

# df = pd.DataFrame()

# for tick,w in zip(tickers,weight):
#   ticker = yf.Ticker(tick)
#   temp = ticker.history(period='5y')
#   temp['PAV'] = temp['Close']*temp['Volume']*w
#   temp.reset_index()
#   df = pd.concat([df,temp])

# df = df.reset_index()
# df = df[['Date','PAV']].groupby('Date').sum().reset_index()
# df

# ticker = yf.Ticker('NDQ.AX')
# temp = ticker.history(period='5y')
# temp = temp.reset_index()



In [22]:

#hide_input
# # fig = go.Figure()
# fig = make_subplots(specs=[[{"secondary_y": True}]])

# fig.add_trace(go.Scatter(
#     name='Weight Aj.Flow',
#     mode="markers+lines", x=df["Date"], y=df["PAV"]
# ),secondary_y=False)

# # fig.add_trace(go.Scatter( name= 'mean vol',
# #     mode="lines", x=df["Date"], y=[df.PAV.mean()]*len(df["Date"])
# # ),secondary_y=False)

# fig.add_trace(go.Scatter(
#     name='NDQ',
#     mode="markers+lines", x=temp["Date"], y=temp["Close"]
# ),secondary_y=True)

# fig.update_layout({
# 'plot_bgcolor': 'rgba(0, 0, 0, 0)',
# 'paper_bgcolor': 'rgba(0, 0, 0, 0)',
# })


# fig.update_layout(legend=dict(
#     orientation="h",
#     yanchor="bottom",
#     y=1.02,
#     xanchor="right",
#     x=1
# ))

# fig.update_layout(title="$USD flowing in the holdings of NDQ")
# fig

